In [1]:
# Import Libraries

import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error as msle, mean_squared_error as mse
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import pickle

In [2]:
# Import data

root = "D:/Kaggle/Energy Prediction/Data/"
df_train = pd.read_csv(f"{root}train.csv")
building_metadata = pd.read_csv(f"{root}building_metadata.csv")
weather_train = pd.read_csv(f"{root}weather_train.csv")

In [3]:
# Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
df_train = reduce_mem_usage(df=df_train)
building_metadata = reduce_mem_usage(df=building_metadata)
weather_train = reduce_mem_usage(df=weather_train)

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)


In [5]:
df_train = df_train.merge(building_metadata, on='building_id', how='left')
df_train.head(5)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN


In [6]:
df_train = df_train.merge(weather_train, on=['site_id', 'timestamp'], how='left')
df_train.head(5)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0


In [7]:
#Label Encoding primary use
df_train.primary_use = LabelEncoder().fit_transform(df_train.primary_use)
df_train.head(5)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,0,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,0,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,0,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,0,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,0,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0


In [8]:
#Divide timestamp into 5 features
df_train['timestamp'] = pd.to_datetime(arg=df_train['timestamp'])
df_train['year'] = df_train['timestamp'].dt.year
df_train['month'] = df_train['timestamp'].dt.month
df_train['day'] = df_train['timestamp'].dt.day
df_train['hour'] = df_train['timestamp'].dt.hour
df_train["weekday"] = df_train['timestamp'].dt.weekday
df_train.head(5)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,year,month,day,hour,weekday
0,0,0,2016-01-01,0.0,0,0,7432,2008.0,NaN,25.0,...,20.0,NaN,1019.5,0.0,0.0,2016,1,1,0,4
1,1,0,2016-01-01,0.0,0,0,2720,2004.0,NaN,25.0,...,20.0,NaN,1019.5,0.0,0.0,2016,1,1,0,4
2,2,0,2016-01-01,0.0,0,0,5376,1991.0,NaN,25.0,...,20.0,NaN,1019.5,0.0,0.0,2016,1,1,0,4
3,3,0,2016-01-01,0.0,0,0,23685,2002.0,NaN,25.0,...,20.0,NaN,1019.5,0.0,0.0,2016,1,1,0,4
4,4,0,2016-01-01,0.0,0,0,116607,1975.0,NaN,25.0,...,20.0,NaN,1019.5,0.0,0.0,2016,1,1,0,4


In [9]:
drop_features = ['day', 'month', 'year', 'cloud_coverage', 'precip_depth_1_hr', 'sea_level_pressure', 
        'wind_direction', 'wind_speed', 'floor_count', 'year_built']
df_train.drop(drop_features, axis=1, inplace=True)
df_train.head(5)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,dew_temperature,hour,weekday
0,0,0,2016-01-01,0.0,0,0,7432,25.0,20.0,0,4
1,1,0,2016-01-01,0.0,0,0,2720,25.0,20.0,0,4
2,2,0,2016-01-01,0.0,0,0,5376,25.0,20.0,0,4
3,3,0,2016-01-01,0.0,0,0,23685,25.0,20.0,0,4
4,4,0,2016-01-01,0.0,0,0,116607,25.0,20.0,0,4


In [10]:
# Missing value handling
cols = ['air_temperature','dew_temperature']
df_train.loc[:, cols] = df_train.loc[:, cols].interpolate(axis=0)

In [11]:
#Drop Null values
df_train.dropna(inplace=True)
df_train.reset_index(inplace = True, drop = True)
df_train.head(5)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,dew_temperature,hour,weekday
0,0,0,2016-01-01,0.0,0,0,7432,25.0,20.0,0,4
1,1,0,2016-01-01,0.0,0,0,2720,25.0,20.0,0,4
2,2,0,2016-01-01,0.0,0,0,5376,25.0,20.0,0,4
3,3,0,2016-01-01,0.0,0,0,23685,25.0,20.0,0,4
4,4,0,2016-01-01,0.0,0,0,116607,25.0,20.0,0,4


In [12]:
#Drop timestamp
drop_features = ["timestamp"]
df_train.drop(drop_features, axis=1, inplace=True)
df_train.head(5)

,building_id,meter,meter_reading,site_id,primary_use,square_feet,air_temperature,dew_temperature,hour,weekday
0,0,0,0.0,0,0,7432,25.0,20.0,0,4
1,1,0,0.0,0,0,2720,25.0,20.0,0,4
2,2,0,0.0,0,0,5376,25.0,20.0,0,4
3,3,0,0.0,0,0,23685,25.0,20.0,0,4
4,4,0,0.0,0,0,116607,25.0,20.0,0,4


In [16]:
model = RandomForestRegressor(n_estimators = 10, random_state = 0)
model.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [ ]:
r_sq = model.score(x_train, y_train)
print('coefficient of determination:', r_sq)

In [ ]:
feat_importances = pd.Series(model.feature_importances_, index=df_train.columns)
feat_importances.plot(kind='barh')

In [13]:
len(df_train)

20216100

In [ ]:
#Save pandas dataframe
df_train.to_csv(f"{root}altered_train.csv", index = False)